In [3]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

In [4]:
#nineth_file = pd.read_excel('train_df.xlsx', sheet_name='Sheet1')
#nineteen_march = pd.read_excel(r'Testing\19_03_2018_Duplicate_Check_Output.xlsx', sheet_name='Duplicates')
#twentieth_march = pd.read_excel(r'Testing\20_03_2018_Duplicate_Check_Output.xlsx',sheet_name='Duplicates')
#twentifirst_march = pd.read_excel(r'Testing\21_03_2018_Duplicate_Check_Output.xlsx',sheet_name='Duplicates')
#twentyseventh_march = pd.read_excel(r'Testing\27-03-2018-Duplicate_Output.xlsx',sheet_name='Duplicates')
second_may = pd.read_excel(r'Testing\02_05_2018_Duplicate_Check_Output.xlsx',sheet_name='Duplicates')
third_may = pd.read_excel(r'Testing\03_05_2018_Duplicate_Check_Output.xlsx',sheet_name='Duplicates')
fifteenth_may = pd.read_excel(r'Testing\15_05_2018_Duplicate_Check_Output.xlsx',sheet_name='Duplicates')

In [6]:
#print(nineteen_march.shape)
#print(twentieth_march.shape)
#print(twentifirst_march.shape)
#print(twentyseventh_march.shape)
print(second_may.shape)
print(third_may.shape)
print(fifteenth_may.shape)

(328, 63)
(324, 63)
(371, 63)


In [7]:
#nineth_file = pd.read_excel('09_02_2018_Duplicate_Check_Output.xlsx', sheet_name='Duplicates')
#fifteenth_file = pd.read_excel('15_02_2018_Duplicate_Check_Output.xlsx', sheet_name='Duplicates')
#dup_test_2 = nineth_file.append(fifteenth_file,ignore_index=True)
dup_test_2 = fifteenth_may

In [8]:
dup_test_2.shape

(371, 63)

In [9]:
dup_test_2.tail()

,u_task u_requested_by,u_task u_requested_by u_area,u_task u_requested_by department,u_task u_requested_by u_service_line,u_task u_requested_by vip,u_task u_requested_by preferred_language,u_task active,u_task contact_type,u_task number,u_task opened_at,...,u_gsd_level_1_fcr,u_gsd_l2_fcr,u_gsd_l1_fcr,u_dispatch_time_l2,FileName,u_task u_survey_sent,u_task parent,Comments,Index,Exceptions
366,Zahid Aslam,EM,United Arab Emirates,Assurance,False,English,True,Phone,SR12103493,10-05-2018 06:36:40,...,NaN,False,False,NaN,Operational SLA extract (still active) - Part ...,NaN,NaN,Dupe - u_task u_business_service - Matching,D,NaN
367,Ze Yin Wang,AP,China,Assurance,False,English,True,Phone,SR12109910,10-05-2018 21:16:04,...,NaN,False,False,NaN,Operational SLA extract (still active) - Part ...,NaN,NaN,Master,O,NaN
368,Ze Yin Wang,AP,China,Assurance,False,English,True,Phone,SR12110141,10-05-2018 21:59:01,...,NaN,False,False,NaN,Operational SLA extract (still active) - Part ...,NaN,NaN,Dupe - u_task u_impacted_application - Matching,D,NaN
369,Zhi Zhi Han,AP,China,Assurance,False,English,True,Phone,SR12092137,09-05-2018 08:18:53,...,NaN,False,False,NaN,Operational SLA extract (still active) - Part ...,NaN,NaN,Master,O,NaN
370,Zhi Zhi Han,AP,China,Assurance,False,English,True,Phone,SR12115822,11-05-2018 07:31:00,...,NaN,False,False,NaN,Operational SLA extract (still active) - Part ...,NaN,NaN,Dupe - u_task u_impacted_application - Matching,D,NaN


In [10]:
dup_test_2.columns

Index(['u_task u_requested_by', 'u_task u_requested_by u_area',
       'u_task u_requested_by department',
       'u_task u_requested_by u_service_line', 'u_task u_requested_by vip',
       'u_task u_requested_by preferred_language', 'u_task active',
       'u_task contact_type', 'u_task number', 'u_task opened_at',
       'u_task u_reopen_count', 'u_task reassignment_count', 'u_task priority',
       'u_task u_resolved_date', 'u_task sys_mod_count',
       'u_task u_business_service', 'u_task u_impacted_application',
       'u_task state', 'u_task short_description', 'u_task cmdb_ci',
       'u_task u_impacted_ci', 'u_task subcategory',
       'u_task u_impacted_account', 'u_task close_notes', 'u_task made_sla',
       'u_task assignment_group u_tier_level', 'u_task u_level1',
       'u_task sys_updated_on', 'u_task sys_updated_by', 'u_task u_level4',
       'u_task u_level3', 'u_task u_level2', 'u_task opened_by',
       'u_task opened_by u_gpn', 'u_task opened_by u_hire_date',
     

In [11]:
dup_test_2.columns = [c.replace(' ','_') for c in dup_test_2.columns]
dup_test_2.columns

Index(['u_task_u_requested_by', 'u_task_u_requested_by_u_area',
       'u_task_u_requested_by_department',
       'u_task_u_requested_by_u_service_line', 'u_task_u_requested_by_vip',
       'u_task_u_requested_by_preferred_language', 'u_task_active',
       'u_task_contact_type', 'u_task_number', 'u_task_opened_at',
       'u_task_u_reopen_count', 'u_task_reassignment_count', 'u_task_priority',
       'u_task_u_resolved_date', 'u_task_sys_mod_count',
       'u_task_u_business_service', 'u_task_u_impacted_application',
       'u_task_state', 'u_task_short_description', 'u_task_cmdb_ci',
       'u_task_u_impacted_ci', 'u_task_subcategory',
       'u_task_u_impacted_account', 'u_task_close_notes', 'u_task_made_sla',
       'u_task_assignment_group_u_tier_level', 'u_task_u_level1',
       'u_task_sys_updated_on', 'u_task_sys_updated_by', 'u_task_u_level4',
       'u_task_u_level3', 'u_task_u_level2', 'u_task_opened_by',
       'u_task_opened_by_u_gpn', 'u_task_opened_by_u_hire_date',
     

## Calculate Similarities

In [12]:
import os
import pandas as pd
import nltk
import gensim
from gensim import corpora, models, similarities

C:\Users\madhan.s\AppData\Local\Continuum\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [13]:
from difflib import SequenceMatcher
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

### Based on Original and Duplicate column

In [14]:
c=0
l = len(dup_test_2)
dup_test_2['sim_value'] = ''
while c<l-1:
    for i in dup_test_2.iterrows():
        if dup_test_2['Index'][c] == 'O':
            text_1 = dup_test_2['u_task_short_description'][c]
            #print("T1:",text_1)
            c=c+1
        else:
            text_2 = dup_test_2['u_task_short_description'][c]
            dup_test_2['sim_value'][c] = round(similar(text_1,text_2),2)
            #print("T2:",text_2)
            #print(similar(text_1,text_2))
            c=c+1

C:\Users\madhan.s\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [29]:
dup_test_2.to_excel(r'Testing\dup_test_2(2nd_may_new).xlsx')

In [15]:
%%time
c = 0
l = len(dup_test_2)
simtest = []
texts=[]
username = []
simtest.append('')
dup_test_2['sim_value'] = ''
for i in dup_test_2.iterrows():
    while c<l-1:
        if dup_test_2['u_task_u_impacted_ci'][c] == dup_test_2['u_task_u_impacted_ci'][c+1]:
            #username.append(dup['u_task_u_requested_by'][c])
            #username.append(dup['u_task_u_requested_by'][c+1])
            text_1 = dup_test_2['u_task_short_description'][c]
            text_2 = dup_test_2['u_task_short_description'][c+1]
            dup_test_2['sim_value'][c] = round(similar(text_1,text_2),2)
            dup_test_2['sim_value'][c+1] = round(similar(text_1,text_2),2)
            #simtest.append(float(similar(text_1,text_2)))
            #simtest.append(' ')
            c=c+1
            texts.append(text_1)
            texts.append(text_2)
            #print("text_1:", text_1)
            #print("text_2:", text_2)
        else:
            c=c+1

#len(simtest)
#len(username)

C:\Users\madhan.s\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\madhan.s\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


Wall time: 1min 23s


## c = 0
l = len(dup_test_2)
simtest = []
texts=[]
username = []
simtest.append('')
dup_test_2['sim_value'] = ''
for i in dup_test_2.iterrows():
    while c<l-1:
        if dup_test_2['u_task_u_impacted_ci'][c] == dup_test_2['u_task_u_impacted_ci'][c+1]:
            if dup_test_2['Exceptions'].any() == 'E':
                dup_test_2['sim_value'][c] = 'Exception'
            else:
                #print(dup_test_2['u_task_short_description'][c])
                #print(dup_test_2['u_task_short_description'][c+1])
                c=c+1
        else:
            c=c+1

In [1]:
#dup_test_2.head()

In [17]:
%%time
c = 0
l = len(dup_test_2)
dup_test_2['threshold_sim_value'] = ''
for i in dup_test_2.iterrows():
    while c<l:
        if (dup_test_2['sim_value'][c] < 0.5):
            dup_test_2['threshold_sim_value'][c] = 'Less than 0.5 threshold'
        else:
            dup_test_2['threshold_sim_value'][c] = 'greater than 0.5 threshold'
        c=c+1

C:\Users\madhan.s\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\madhan.s\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


Wall time: 1min 14s


In [18]:
df_lessthan  = dup_test_2[dup_test_2.threshold_sim_value =='Less than 0.5 threshold']
df_greaterthan = dup_test_2[dup_test_2.threshold_sim_value =='greater than 0.5 threshold']
print(df_lessthan.shape)
print(df_greaterthan.shape)
print(dup_test_2.shape)

(165, 65)
(206, 65)
(371, 65)


In [19]:
writer = pd.ExcelWriter(r'Testing\dup_test_2(15th_may).xlsx', engine='xlsxwriter')

In [20]:
dup_test_2.to_excel(writer, sheet_name='all tickets')
df_lessthan.to_excel(writer, sheet_name='< 0.5 threshold')
df_greaterthan.to_excel(writer, sheet_name='> 0.5 threshold')
writer.save()